In [ ]:
# Author: Naveen Lalwani
# Script to convert model from .h5 to .tflite; post train quantize models and evaluate performance for LeNet-300-100 on either dataset

import tensorflow as tf
import keras
import numpy as np
import time
from tensorflow.contrib import lite
from keras.utils import np_utils
import os

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", reshape = False, one_hot=True)

In [ ]:
numTest = mnist.test.images.shape[0]
testData = mnist.test.images[:numTest,:]
testLabels = mnist.test.labels[:numTest,:]
print(numTest)

Code for loading keras model and converting it to tflite model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(FILEPATH/FILENAME.h5) #"C://users/navee/Folder1/after_pruning_conv2_retrain.h5"
tflite_model = converter.convert()
open(FILENAME.tflite, "wb").write(tflite_model) #"Before_Pruning.tflite"

In [ ]:
interpreter = tf.lite.Interpreter(model_path=FILEPATH/FILENAME.tflite)# "C://users/navee/Folder1/BeforePruning_MNIST.tflite"
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
start = time.time()
count = 0
for i in range(numTest):
    input_data = np.array(testData[i].reshape([1, 28, 28]), dtype = np.float32)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    output_details = interpreter.get_output_details()
    output_index = output_details[0]['index']
    output_data = interpreter.get_tensor(output_index)
    pred_model = np.argmax(output_data)
    pred_data = np.argmax(testLabels[i])
    if(pred_model == pred_data):
        count += 1
end = time.time()
print("Inference Time: " + str(end - start))
print("Test Accuracy = " + (str((count / numTest) * 100)))

Quantization of model and saving

In [ ]:
'''
Comment these lines after running the files once and take readings for the original model and quantized model.
'''
converter.post_training_quantize=True
tflite_quantized_model=converter.convert()
open(FILENAME.tflite, "wb").write(tflite_quantized_model) #"quantized_model.tflite"

In [ ]:
interpreter = tf.lite.Interpreter(model_path=FILEPATH/FILENAME.tflite) #"C://users/navee/Folder1/quantized_model.tflite"
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
start = time.time()
count = 0
for i in range(numTest):
    input_data = np.array(testData[i].reshape([1, 28, 28]), dtype = np.float32)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    output_details = interpreter.get_output_details()
    output_index = output_details[0]['index']
    output_data = interpreter.get_tensor(output_index)
    pred_model = np.argmax(output_data)
    pred_data = np.argmax(testLabels[i])
    if(pred_model == pred_data):
        count += 1
end = time.time()
print("Inference Time: " + str(end - start))
print("Test Accuracy = " + (str((count / numTest) * 100)))